In [3]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import pandas as pd
import glob
import pickle
import numpy as np
from csslconstants import *
from cmcrameri import cm

homedir = '/home/marianne/Documents/cssltimeseries/'
datadir = homedir + 'data/' 
os.chdir(homedir)

## load data from the prep_cssl_data.py process
with open(datadir + 'cssldata.pickle', 'rb') as handle:
    cssl = pickle.load(handle)


In [4]:
## get CSSL  monthly data
csslmonthly = pd.read_csv(datadir + 'csslmonthly.csv')

In [5]:
csslmonthly

,Date,Air Temp Max (C),Air Temp Min (C),24-hour Total Precip (mm),Season Total Precip (mm),% of Precip as Snow,% of Precip as Rain,New Snow (cm),Season Total Snow (cm),Snowpack depth (cm),Snow Water Equivalent (cm),dt
0,1970-10-31,13.176774,-0.179032,2.580968,25.899806,NaN,NaN,1.651000,14.584516,7.292258,0.000000,2.488320e+16
1,1970-11-30,5.938500,-2.053500,15.820571,273.658067,NaN,NaN,5.533571,79.048667,29.210000,16.002000,2.751840e+16
2,1970-12-31,-0.537097,-8.718871,14.264968,803.749871,NaN,NaN,18.216880,494.402452,203.044612,51.056988,3.015360e+16
3,1971-01-31,3.562742,-7.895323,6.677742,1092.049548,NaN,NaN,5.649310,762.260323,232.542976,73.754129,3.283200e+16
4,1971-02-28,4.261607,-8.661964,2.313214,1186.632214,NaN,NaN,1.164167,833.760000,181.676842,73.847158,3.538080e+16
...,...,...,...,...,...,...,...,...,...,...,...,...
583,2019-05-31,11.677419,-0.967742,5.645161,2038.645161,87.25,73.466667,1.870968,1203.290323,183.758065,102.513333,1.557965e+18
584,2019-06-30,19.833333,3.633333,0.233333,2148.366667,NaN,100.000000,0.000000,1239.500000,36.750000,15.929167,1.560600e+18
585,2019-07-31,22.935484,7.129032,0.000000,2150.000000,NaN,NaN,0.000000,1239.500000,0.000000,0.000000,1.563235e+18
586,2019-08-31,24.129032,7.935484,0.000000,2150.000000,NaN,100.000000,0.000000,1239.500000,0.000000,0.000000,1.565914e+18
